In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

e:\ML Projects\Autogen-AgentAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override = True)
client = OpenAI()

In [3]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = "https://api.pushover.net/1/messages.json"

In [5]:
def push(message):
    print(f"Push: {message}")
    payload = {"user" : pushover_user, "token" : pushover_token, "message" : message}
    requests.post(pushover_url, data=payload)

In [6]:
push("Hey!")

Push: Hey!


In [8]:
def record_user_details(email, name = "Name not provided", notes = "Not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [9]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I could not answer")
    return {"recorded": "ok"}

This is a tool

In [10]:
record_user_details_json = {
    "name" : "record_user_details",
    "description" : "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "email" : {
                "type" : "string",
                "description" : "The email address this user"
            },
            "name" : {
                "type" : "string",
                "description" : "The user's name, if they provided it"
            },
            "notes" : {
                "type" : "string",
                "description" : "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required" : ['email'],
        "additionalProperties" : False
    }
}

In [11]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [12]:
tools = [{"type" : "function", "function" : record_user_details_json},
         {"type" : "function", "function" : record_unknown_question_json}]

In [13]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question

In [14]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.load(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
    return result

In [15]:
globals()["record_unknown_question"]("This is a really hard question")

Push: Recording This is a really hard question asked that I could not answer


{'recorded': 'ok'}

In [16]:
# Writing the above function in a more elegant way 